In [10]:
# imports

import os
import requests
#from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
#from openai import OpenAI
import ollama
# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [11]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [12]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

In [13]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [14]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [15]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

In [16]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [17]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

In [19]:
MODEL = "llama3.2"

In [20]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = ollama.chat(
        model = MODEL,
        messages = messages_for(website)
    )
    return response['message']['content']

In [21]:
summarize("https://edwarddonner.com")

'**Summary**\n================\n\n* Edward Donner is the founder and CTO of Nebula.io, a company applying AI to help people discover their potential.\n* He has co-founded and worked with various AI startups, including untapt, which was acquired in 2021.\n\n**News/Announcements**\n--------------------\n\n* **December 21, 2024**: LLM Workshop – Hands-on with Agents - resources\n* **November 13, 2024**: Welcome, SuperDataScientists!\n* **October 16, 2024**: From Software Engineer to AI Data Scientist – resources\n* **January 23, 2025**: LLM Workshop (upcoming event)\n\n**No navigation content has been included in this summary.**'

In [22]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [23]:
display_summary("https://edwarddonner.com")

### Website Summary

#### Overview
The website is owned by Edward Donner, the co-founder and CTO of Nebula.io, a company applying AI to help people discover their potential. The website appears to be a personal blog or portfolio site.

#### Content Highlights

* **News and Announcements**
	+ Upcoming LLM Workshop on January 23, 2025
	+ Previous workshops: December 21, 2024 (LLM Workshop – Hands-on with Agents), November 13, 2024 (Welcome, SuperDataScientists!), October 16, 2024 (Mastering AI and LLM Engineering)
* **Personal Interests**
	+ DJing and amateur electronic music production
	+ Hacker News enthusiast
* **Professional Background**
	+ Co-founder and CTO of Nebula.io
	+ Previously founder and CEO of AI startup untapt (acquired in 2021)

#### Links

* LinkedIn
* Twitter
* Facebook
* Newsletter subscription

In [24]:
display_summary("https://cnn.com")

This appears to be the front page of CNN.com, the website for the American cable news channel CNN (Cable News Network). The page is constantly updated with breaking news stories, videos, and analysis on various topics including politics, business, entertainment, sports, and more.

Here are some notable features and sections on this page:

1. **Headlines**: A section featuring the top news headlines from around the world.
2. **Videos**: A collection of short videos related to current events, interviews with newsmakers, and analysis segments.
3. **News**: A section that aggregates breaking news stories from various parts of the world.
4. **Politics**: A section dedicated to US politics, including presidential elections, legislative updates, and opinion pieces.
5. **Business**: A section covering business news, markets, and economic trends.
6. **Entertainment**: A section featuring entertainment news, reviews, and analysis on movies, TV shows, music, and celebrities.
7. **Sports**: A section dedicated to sports news, scores, and updates on various professional leagues.
8. **Features**: A section with in-depth articles, interviews, and profiles on a wide range of topics.
9. **Opinion**: A section featuring opinion pieces from CNN's editorial team and other writers on current events and issues.

This is just one example of the many ways that CNN presents news to its audience.

In [25]:
display_summary("https://anthropic.com")

# Website Summary
### Title: Just a Moment...

**Overview**
The website appears to be under maintenance, requiring users to enable JavaScript and cookies to proceed. No content is displayed until these conditions are met.

### News/Announcements (none found)
Unfortunately, there are no news or announcements on this website at the time of analysis.